In [2]:
import os
import json
import pandas as pd
import traceback

In [5]:
from langchain_openai import ChatOpenAI

In [6]:
from dotenv import load_dotenv
load_dotenv() # Load environment variables from .env file

True

In [7]:
llm = ChatOpenAI(model="gpt-4o", temperature=0.0, max_tokens=1000, api_key=os.getenv("OPENAI_API_KEY"))

In [10]:
from langchain_openai import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2




In [11]:
response_json= {
    "1":{
        "mcq": "multiple choice questiom",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here"
        },
        "correct": "correct answer here"
    },
     "2":{
        "mcq": "multiple choice questiom",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here"
        },
        "correct": "correct answer here"

    },
     "3":{
        "mcq": "multiple choice questiom",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here"
        },
        "correct": "correct answer here"

    }
}

TEMPLATE= """text: {text}
You are an expert MCQ generator. Given the  above text, it is your job to create a quiz of {number} multiple \
     choice questions for {subject} students in {tone} tone.
     Make sure the questions are not repeated and check all the questions to be confirming to text as well.
     Make sure to format your response like RESPONSE_JSON below and use it as a guide. \
    Ensure to make {number} questions.
    ### RESPONSE_JSON
    # {response_json}
    ### END_RESPONSE_JSON"""

In [12]:
quiz_generation_prompt = PromptTemplate(
    input_variables=['text','number','subject','tone','response_json'],
    template= TEMPLATE
)

In [13]:
quiz_chain = LLMChain(llm=llm, prompt= quiz_generation_prompt, output_key="quiz", verbose=True)


/var/folders/nq/9bf8jjjn7nvc7d7_v7j998zc0000gn/T/ipykernel_7426/2153818086.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  quiz_chain = LLMChain(llm=llm, prompt= quiz_generation_prompt, output_key="quiz", verbose=True)


In [15]:
TEMPLATE2="""You are an expert english grammarian and writer. Given a Multiple choice quiz for {subject} students. \
     You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use around 50 words for complexity \
     if the quiz is not at per the cognitive and analytical abilties of the students,\
     update the quiz questions, which need to be changed and change the tone such that it perfectly fits the student's abilties.
     Quiz MCQs:
     {quiz}

     Check from an expert english writer of the above quiz:
          """

In [16]:
quiz_evaluation_prompt = PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE2)

In [17]:
review_chain = LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [21]:
generate_evaluate_chain = SequentialChain(
    chains=[quiz_chain, review_chain],
    input_variables=["text", "number", "subject", "tone", "response_json"],
    output_variables=["quiz", "review"],    
    verbose=True)

In [20]:
file_path =r"/Users/abdullahmahmud/Desktop/Ali_Work/GenAI Course/mcq_gen/data.txt"
with open(file_path, 'r') as file:
    TEXT = file.read()

In [23]:
NUMBER = 3
SUBJECT = "Machine Learning"
TONE = "simple"


In [24]:
with get_openai_callback() as cb: #for tracking token usage
    response= generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject": SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(response_json)  # Ensure response_json is a string
        }
    )

/var/folders/nq/9bf8jjjn7nvc7d7_v7j998zc0000gn/T/ipykernel_7426/3357549021.py:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  response= generate_evaluate_chain(
Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")
Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


Prompt after formatting:
text: Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalise to unseen data, and thus perform tasks without explicit instructions.[1] Within a subdiscipline in machine learning, advances in the field of deep learning have allowed neural networks, a class of statistical algorithms, to surpass many previous machine learning approaches in performance.[2]

ML finds application in many fields, including natural language processing, computer vision, speech recognition, email filtering, agriculture, and medicine.[3][4] The application of ML to business problems is known as predictive analytics.

Statistics and mathematical optimisation (mathematical programming) methods comprise the foundations of machine learning. Data mining is a related field of study, focusing on exploratory data analysis (EDA) via unsupervised learning.[6][7]

From a theoretica

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")



> Finished chain.
Prompt after formatting:
You are an expert english grammarian and writer. Given a Multiple choice quiz for Machine Learning students.      You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use around 50 words for complexity      if the quiz is not at per the cognitive and analytical abilties of the students,     update the quiz questions, which need to be changed and change the tone such that it perfectly fits the student's abilties.
     Quiz MCQs:
     ```json
{
    "1": {
        "mcq": "What is the primary focus of machine learning?",
        "options": {
            "a": "To create symbolic AI systems",
            "b": "To develop statistical algorithms that learn from data",
            "c": "To replace human intelligence",
            "d": "To build expert systems"
        },
        "correct": "b"
    },
    "2": {
        "mcq": "Who coined the term 'machine learning'?",
        "options": {
            "a": 

In [25]:
print(f"Token Usage: {cb.total_tokens} ")
print(f"Prompt Tokens: {cb.prompt_tokens}")
print(f"Completion Tokens: {cb.completion_tokens}")
print(f"Total Cost: ${cb.total_cost:.4f}")

Token Usage: 3106 
Prompt Tokens: 2550
Completion Tokens: 556
Total Cost: $0.0119


In [26]:
response

{'text': 'Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalise to unseen data, and thus perform tasks without explicit instructions.[1] Within a subdiscipline in machine learning, advances in the field of deep learning have allowed neural networks, a class of statistical algorithms, to surpass many previous machine learning approaches in performance.[2]\n\nML finds application in many fields, including natural language processing, computer vision, speech recognition, email filtering, agriculture, and medicine.[3][4] The application of ML to business problems is known as predictive analytics.\n\nStatistics and mathematical optimisation (mathematical programming) methods comprise the foundations of machine learning. Data mining is a related field of study, focusing on exploratory data analysis (EDA) via unsupervised learning.[6][7]\n\nFrom a theoretical viewpoint, pr

In [36]:
quiz = response['quiz'].strip().strip('```json').strip('```').strip()


In [39]:
quiz=json.loads(quiz)  # Assuming quiz is a JSON string, convert it to a Python dictionary


In [40]:
quiz_table_data=[]
for key, value in quiz.items():
    mcq = value['mcq']
    options = "|".join([f"{option}: {option_value}" for option, option_value in value['options'].items()])

    correct = value['correct']
    quiz_table_data.append({
        "MCQ": mcq,
        "Choices": options,
        "Correct": correct})

In [41]:
quiz_table_data

[{'MCQ': 'What is the primary focus of machine learning?',
  'Choices': 'a: To create symbolic AI systems|b: To develop statistical algorithms that learn from data|c: To replace human intelligence|d: To build expert systems',
  'Correct': 'b'},
 {'MCQ': "Who coined the term 'machine learning'?",
  'Choices': 'a: Alan Turing|b: Geoffrey Hinton|c: Arthur Samuel|d: Donald Hebb',
  'Correct': 'c'},
 {'MCQ': 'Which of the following is a subset of machine learning?',
  'Choices': 'a: Artificial intelligence|b: Deep learning|c: Data mining|d: Predictive analytics',
  'Correct': 'b'}]

In [43]:
quiz=pd.DataFrame(quiz_table_data)  # Convert the list of dictionaries to a DataFrame

In [44]:
quiz.to_csv("quiz.csv", index=False)  # Save the DataFrame to a CSV file